In [4]:
import xarray as xr

ds_inst = xr.open_dataset("data/Beed/Beed_2015_01/data_stream-oper_stepType-instant.nc")
print(ds_inst.data_vars)


Data variables:
    msl      (valid_time, latitude, longitude) float32 12kB ...
    u10      (valid_time, latitude, longitude) float32 12kB ...
    v10      (valid_time, latitude, longitude) float32 12kB ...


In [5]:
ds_acc = xr.open_dataset("data/Beed/Beed_2015_01/data_stream-oper_stepType-accum.nc")
print(ds_acc.data_vars)

Data variables:
    strd     (valid_time, latitude, longitude) float32 12kB ...
    ssrd     (valid_time, latitude, longitude) float32 12kB ...


In [9]:
import xarray as xr
import pandas as pd
import numpy as np

def process_one_month(year, month):
    month_str = f"{month:02d}"

    base_path = f"data/Beed/Beed_{year}_{month_str}"
    instant_path = f"{base_path}/data_stream-oper_stepType-instant.nc"
    accum_path   = f"{base_path}/data_stream-oper_stepType-accum.nc"
    rh_path      = f"data_pressure/Beed/Beed_rh1000_{year}_{month_str}.nc"

    # ================= INSTANT =================
    ds_inst = xr.open_dataset(instant_path)

    lats = ds_inst.latitude.values
    lons = ds_inst.longitude.values

    city_lat = 18.9901
    city_lon = 75.7531

    def haversine(lat1, lon1, lat2, lon2):
     R = 6371
     lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lat2, lat2, lon2])
     dlat = lat2 - lat1
     dlon = lon2 - lon1
     a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
     return 2 * R * np.arcsin(np.sqrt(a))

    distances = np.zeros((len(lats), len(lons)))

    for i, lat in enumerate(lats):
     for j, lon in enumerate(lons):
      distances[i, j] = haversine(city_lat, city_lon, lat, lon)

    p = 2
    weights = 1 / (distances ** p)
    weights = weights / weights.sum()

    idw_results = {}

    for var in ds_inst.data_vars:
       print(f"Processing variable: {var}")

       data = ds_inst[var].values   # shape: (time, 2, 2)
       idw_series = np.sum(data * weights, axis=(1, 2))

       idw_results[var] = idw_series

    u10 = idw_results["u10"]
    v10 = idw_results["v10"]

    wind_speed = np.sqrt(u10**2 + v10**2)  

    time = pd.to_datetime(ds_inst.valid_time.values)

    df_hourly = pd.DataFrame({
       "datetime": time,
       "msl": idw_results["msl"],
       "wind_speed": wind_speed
    }) 

    # ================= ACCUM =================
    ds_acc = xr.open_dataset(accum_path)
    idw_acc = {}

    for var in ds_acc.data_vars:
       data = ds_acc[var].values
       idw_series = np.sum(data * weights, axis=(1, 2))
       idw_acc[var] = idw_series

    df_hourly["datetime"] = pd.to_datetime(df_hourly["datetime"])
    df_hourly = df_hourly.set_index("datetime")   
    df_daily = df_hourly.resample("D").mean()
    time_acc = pd.to_datetime(ds_acc.valid_time.values)

    df_acc_hourly = pd.DataFrame({
       "datetime": time_acc,
    **idw_acc
    })
    df_acc_hourly["datetime"] = pd.to_datetime(df_acc_hourly["datetime"])
    df_acc_hourly = df_acc_hourly.set_index("datetime")

    df_acc_daily = df_acc_hourly.resample("D").sum()
    df_acc_daily = df_acc_daily.rename(columns={
      "ssrd": "solar_radiation",
      "strd": "longwave_radiation"
    })
    df_merged = df_daily.join(df_acc_daily, how="inner")
    df_merged = df_merged.reset_index().rename(columns={"index": "date"})
    df_merged["datetime"] = pd.to_datetime(df_merged["datetime"])
    df_merged = df_merged.set_index("datetime")

    # ================= RH (PRESSURE LEVEL) =================
    ds_rh = xr.open_dataset(rh_path)
    rh = ds_rh["r"].sel(pressure_level=1000).values
    rh_idw = np.sum(rh * weights, axis=(1, 2))
    time_rh = pd.to_datetime(ds_rh.valid_time.values)

    df_rh_hourly = pd.DataFrame({
       "datetime": time_rh,
       "relative_humidity": rh_idw
    })
    df_rh_hourly["datetime"] = pd.to_datetime(df_rh_hourly["datetime"])
    df_rh_hourly = df_rh_hourly.set_index("datetime")

    df_rh_daily = df_rh_hourly.resample("D").mean()
    df_rh_daily.index = pd.to_datetime(df_rh_daily.index)
    df_final = df_merged.join(df_rh_daily, how="inner")

    return df_final 


In [12]:
df_test = process_one_month(2022, 11)
print(df_test.shape)
print(df_test.head())
print(df_test.index.min(), df_test.index.max())

Processing variable: msl
Processing variable: u10
Processing variable: v10
(30, 5)
                      msl  wind_speed  longwave_radiation  solar_radiation  \
datetime                                                                     
2022-11-01  101494.967420    2.117769        2.963491e+07     2.106423e+07   
2022-11-02  101367.634036    2.081452        3.097465e+07     1.923251e+07   
2022-11-03  101400.366734    2.617290        2.876696e+07     2.087271e+07   
2022-11-04  101496.847557    2.670120        2.838026e+07     2.091021e+07   
2022-11-05  101620.436282    2.401128        2.809774e+07     2.113590e+07   

            relative_humidity  
datetime                       
2022-11-01          60.529547  
2022-11-02          65.783408  
2022-11-03          56.197776  
2022-11-04          49.095031  
2022-11-05          49.362665  
2022-11-01 00:00:00 2022-11-30 00:00:00


In [13]:
df_test.head()

,msl,wind_speed,longwave_radiation,solar_radiation,relative_humidity
datetime,,,,,
2022-11-01,101494.967420,2.117769,2.963491e+07,2.106423e+07,60.529547
2022-11-02,101367.634036,2.081452,3.097465e+07,1.923251e+07,65.783408
2022-11-03,101400.366734,2.617290,2.876696e+07,2.087271e+07,56.197776
2022-11-04,101496.847557,2.670120,2.838026e+07,2.091021e+07,49.095031
2022-11-05,101620.436282,2.401128,2.809774e+07,2.113590e+07,49.362665


In [14]:
all_dfs = []

for year in range(2015, 2025):          # 2015 to 2024 inclusive
    for month in range(1, 13):          # Jan to Dec
        print(f"Processing {year}-{month:02d}...")

        try:
            df_month = process_one_month(year, month)

            # Safety check
            if df_month is None or df_month.empty:
                print(f"⚠️ Empty data for {year}-{month:02d}, skipping")
                continue

            all_dfs.append(df_month)

        except Exception as e:
            print(f"❌ Failed for {year}-{month:02d}: {e}")


Processing 2015-01...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-02...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-03...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-04...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-05...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-06...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-07...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-08...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-09...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-10...
Processing variable: msl
Processing variable: u10
Processing variable: v10
Processing 2015-11...
Processi

In [15]:
df_10yr = pd.concat(all_dfs).sort_index()

In [16]:
print(df_10yr.shape)
print(df_10yr.index.min(), df_10yr.index.max())
print(df_10yr.isna().sum())

(3653, 5)
2015-01-01 00:00:00 2024-12-31 00:00:00
msl                   0
wind_speed            0
longwave_radiation    0
solar_radiation       0
relative_humidity     0
dtype: int64


In [17]:
df_10yr.to_csv("beed_2015_2024_era5.csv")

In [56]:
tmax_path = "IMD_maxT_yearwise/Maxtemp_MaxT_2015.GRD"
raw = np.fromfile(tmax_path, dtype=np.float32)
print(raw.size)

350765


In [57]:
import numpy as np

def load_imd_tmax_year(file_path):
    raw = np.fromfile(file_path, dtype=np.float32)
    data = raw.reshape(365, 31, 31)
    return data


In [58]:
lats = np.arange(7.5, 38.5, 1.0)   # 7.5, 8.5, ..., 37.5  → 31 values
lons = np.arange(67.5, 98.5, 1.0)  # 67.5, 68.5, ..., 97.5 → 31 values

print(len(lats), len(lons))  # should be 31 31

31 31


In [59]:
beed_lat = 18.99
beed_lon = 75.76

lat_idx = np.argmin(np.abs(lats - beed_lat))
lon_idx = np.argmin(np.abs(lons - beed_lon))

print(lat_idx, lon_idx, lats[lat_idx], lons[lon_idx])


11 8 18.5 75.5


In [60]:
def extract_beed_tmax_series(tmax_data, lat_idx, lon_idx):
    series = tmax_data[:, lat_idx, lon_idx]
    return series

In [61]:
import pandas as pd

tmax_path_2015 = r"IMD_maxT_yearwise/Maxtemp_MaxT_2015.GRD"

tmax_2015 = load_imd_tmax_year(tmax_path_2015)
beed_tmax_2015 = extract_beed_tmax_series(tmax_2015, lat_idx, lon_idx)

dates_2015 = pd.date_range(start="2015-01-01", end="2015-12-31", freq="D")

df_2015 = pd.DataFrame({
    "date": dates_2015,
    "tmax": beed_tmax_2015
}).set_index("date")

print(df_2015.head())
print(df_2015.shape)


                 tmax
date                 
2015-01-01  24.916723
2015-01-02  26.796511
2015-01-03  27.615252
2015-01-04  26.975063
2015-01-05  28.338764
(365, 1)


In [63]:
import numpy as np
import calendar

def load_imd_tmax_year(file_path, year):
    raw = np.fromfile(file_path, dtype=np.float32)
    
    days = 366 if calendar.isleap(year) else 365
    
    expected = days * 31 * 31
    if raw.size != expected:
        raise ValueError(f"Size mismatch for {year}: got {raw.size}, expected {expected}")
    
    data = raw.reshape(days, 31, 31)
    return data


In [65]:
all_years = []

for year in range(2015, 2025):
    print(f"Processing {year}...")

    path = rf"C:\Users\sid24\era5_download\IMD_maxT_yearwise\Maxtemp_MaxT_{year}.GRD"
    
    tmax_data = load_imd_tmax_year(path, year)

    beed_series = extract_beed_tmax_series(tmax_data, lat_idx, lon_idx)

    dates = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31", freq="D")

    df_year = pd.DataFrame({
        "date": dates,
        "tmax": beed_series
    }).set_index("date")

    all_years.append(df_year)

df_tmax_all = pd.concat(all_years)
print(df_tmax_all.shape)
print(df_tmax_all.head())
print(df_tmax_all.tail())


Processing 2015...
Processing 2016...
Processing 2017...
Processing 2018...
Processing 2019...
Processing 2020...
Processing 2021...
Processing 2022...
Processing 2023...
Processing 2024...
(3653, 1)
                 tmax
date                 
2015-01-01  24.916723
2015-01-02  26.796511
2015-01-03  27.615252
2015-01-04  26.975063
2015-01-05  28.338764
                 tmax
date                 
2024-12-27  28.766357
2024-12-28  31.128965
2024-12-29  30.863199
2024-12-30  31.122478
2024-12-31  30.862686


In [89]:
out_path = r"C:\Users\sid24\era5_download\beed_temperature_2015_2024.csv"
df_tmax_all.to_csv(out_path)

print("Saved to:", out_path)

Saved to: C:\Users\sid24\era5_download\beed_temperature_2015_2024.csv


In [81]:
import numpy as np
import pandas as pd
import os

In [82]:
# IMD 0.25° grid definition
imd_lats = np.linspace(6.5, 38.5, 129)
imd_lons = np.linspace(66.5, 100.0, 135)

# Beed coordinates
beed_lat = 18.99
beed_lon = 75.75

lat_idx = np.argmin(np.abs(imd_lats - beed_lat))
lon_idx = np.argmin(np.abs(imd_lons - beed_lon))

print("Beed grid index:", lat_idx, lon_idx)
print("Grid lat/lon:", imd_lats[lat_idx], imd_lons[lon_idx])


Beed grid index: 50 37
Grid lat/lon: 19.0 75.75


In [83]:
def load_imd_rain_year_auto(file_path, year):
    raw = np.fromfile(file_path, dtype=np.float32)

    # Try 365
    if raw.size == 365 * 129 * 135:
        days = 365
    # Try 366 (leap year)
    elif raw.size == 366 * 129 * 135:
        days = 366
    else:
        raise ValueError(f"{year}: File size {raw.size} does not match 365 or 366 day IMD grid")

    data = raw.reshape(days, 129, 135)
    print(f"{year}: Loaded with {days} days")

    return data, days


In [84]:
def extract_beed_rain_series(rain_data, lat_idx, lon_idx):
    series = []

    for day in range(rain_data.shape[0]):
        window = rain_data[
            day,
            lat_idx-1:lat_idx+2,
            lon_idx-1:lon_idx+2
        ]
        series.append(np.nanmean(window))

    return np.array(series)


In [85]:
base_dir = r"IMD_rainfall_yearwise"

all_years_df = []

In [86]:
for year in range(2015, 2025):
    print(f"\nProcessing {year}...")

    file_path = os.path.join(base_dir, f"Rainfall_ind{year}_rfp25.grd")

    if not os.path.exists(file_path):
        print(f"❌ File missing: {file_path}")
        continue

    # Load data
    rain_data, days = load_imd_rain_year_auto(file_path, year)

    # Extract Beed series
    beed_series = extract_beed_rain_series(rain_data, lat_idx, lon_idx)

    # Build date range safely
    dates = pd.date_range(start=f"{year}-01-01", periods=days, freq="D")

    df_year = pd.DataFrame({
        "date": dates,
        "rainfall": beed_series
    }).set_index("date")

    print(f"{year} shape:", df_year.shape, 
          "| min:", df_year["rainfall"].min(), 
          "| max:", df_year["rainfall"].max())

    all_years_df.append(df_year)



Processing 2015...
2015: Loaded with 365 days
2015 shape: (365, 1) | min: 0.0 | max: 27.541746

Processing 2016...
2016: Loaded with 366 days
2016 shape: (366, 1) | min: 0.0 | max: 102.466896

Processing 2017...
2017: Loaded with 365 days
2017 shape: (365, 1) | min: 0.0 | max: 81.98221

Processing 2018...
2018: Loaded with 365 days
2018 shape: (365, 1) | min: 0.0 | max: 56.306465

Processing 2019...
2019: Loaded with 365 days
2019 shape: (365, 1) | min: 0.0 | max: 42.53359

Processing 2020...
2020: Loaded with 366 days
2020 shape: (366, 1) | min: 0.0 | max: 48.13036

Processing 2021...
2021: Loaded with 365 days
2021 shape: (365, 1) | min: 0.0 | max: 64.001015

Processing 2022...
2022: Loaded with 365 days
2022 shape: (365, 1) | min: 0.0 | max: 40.320343

Processing 2023...
2023: Loaded with 365 days
2023 shape: (365, 1) | min: 0.0 | max: 43.076458

Processing 2024...
2024: Loaded with 366 days
2024 shape: (366, 1) | min: 0.0 | max: 96.161514


In [87]:
df_rain_10yr = pd.concat(all_years_df)

print("\nFINAL DATASET")
print(df_rain_10yr.shape)
print(df_rain_10yr.index.min(), "to", df_rain_10yr.index.max())
print(df_rain_10yr.head())
print(df_rain_10yr.tail())


FINAL DATASET
(3653, 1)
2015-01-01 00:00:00 to 2024-12-31 00:00:00
            rainfall
date                
2015-01-01  2.560208
2015-01-02  4.189449
2015-01-03  0.151907
2015-01-04  0.000000
2015-01-05  0.000000
            rainfall
date                
2024-12-27       0.0
2024-12-28       0.0
2024-12-29       0.0
2024-12-30       0.0
2024-12-31       0.0


In [88]:
out_path = r"C:\Users\sid24\era5_download\beed_rainfall_2015_2024.csv"
df_rain_10yr.to_csv(out_path)

print("Saved to:", out_path)


Saved to: C:\Users\sid24\era5_download\beed_rainfall_2015_2024.csv


In [90]:
era5_path = r"C:\Users\sid24\era5_download\beed_2015_2024_era5.csv"
rain_path = r"C:\Users\sid24\era5_download\beed_rainfall_2015_2024.csv"
tmax_path = r"C:\Users\sid24\era5_download\beed_temperature_2015_2024.csv"

df_era5 = pd.read_csv(era5_path)
df_rain = pd.read_csv(rain_path)
df_tmax = pd.read_csv(tmax_path)


In [91]:
print(df_era5.columns)
print(df_rain.columns)
print(df_tmax.columns)

Index(['datetime', 'msl', 'wind_speed', 'longwave_radiation',
       'solar_radiation', 'relative_humidity'],
      dtype='object')
Index(['date', 'rainfall'], dtype='object')
Index(['date', 'tmax'], dtype='object')


In [92]:
# ERA5
if "datetime" in df_era5.columns:
    df_era5["date"] = pd.to_datetime(df_era5["datetime"])
elif "date" in df_era5.columns:
    df_era5["date"] = pd.to_datetime(df_era5["date"])
else:
    raise ValueError("ERA5 has no date column")

df_era5 = df_era5.set_index("date").drop(columns=[c for c in ["datetime"] if c in df_era5.columns])


In [93]:
# Rainfall
df_rain["date"] = pd.to_datetime(df_rain["date"])
df_rain = df_rain.set_index("date")

In [94]:
# Tmax
df_tmax["date"] = pd.to_datetime(df_tmax["date"])
df_tmax = df_tmax.set_index("date")

In [95]:
print("ERA5:", df_era5.index.min(), "→", df_era5.index.max(), df_era5.shape)
print("RAIN:", df_rain.index.min(), "→", df_rain.index.max(), df_rain.shape)
print("TMAX:", df_tmax.index.min(), "→", df_tmax.index.max(), df_tmax.shape)

ERA5: 2015-01-01 00:00:00 → 2024-12-31 00:00:00 (3653, 5)
RAIN: 2015-01-01 00:00:00 → 2024-12-31 00:00:00 (3653, 1)
TMAX: 2015-01-01 00:00:00 → 2024-12-31 00:00:00 (3653, 1)


In [96]:
df_merge1 = df_era5.join(df_rain, how="inner")
print("After ERA5 + Rain:", df_merge1.shape)

After ERA5 + Rain: (3653, 6)


In [97]:
df_master = df_merge1.join(df_tmax, how="inner")
print("After + Tmax:", df_master.shape)

After + Tmax: (3653, 7)


In [98]:
print(df_master.head())
print(df_master.tail())
print(df_master.isna().sum())

                      msl  wind_speed  longwave_radiation  solar_radiation  \
date                                                                         
2015-01-01  101090.559643    1.207363        3.333532e+07     8.174398e+06   
2015-01-02  101337.271875    1.612628        3.232026e+07     1.123444e+07   
2015-01-03  101495.967796    2.021170        3.195016e+07     1.261227e+07   
2015-01-04  101610.956203    2.790851        3.154876e+07     1.196762e+07   
2015-01-05  101561.523795    2.175815        2.989435e+07     1.741315e+07   

            relative_humidity  rainfall       tmax  
date                                                
2015-01-01          82.239364  2.560208  24.916723  
2015-01-02          79.560310  4.189449  26.796510  
2015-01-03          83.357587  0.151907  27.615252  
2015-01-04          76.794933  0.000000  26.975063  
2015-01-05          68.268212  0.000000  28.338764  
                      msl  wind_speed  longwave_radiation  solar_radiation  \
date

In [99]:
print("Final range:", df_master.index.min(), "→", df_master.index.max())

Final range: 2015-01-01 00:00:00 → 2024-12-31 00:00:00


In [ ]:
out_path = r"C:\Users\sid24\era5_download\beed_master_2015_2024.csv"
df_master.to_csv(out_path)

print("Final master dataset saved to:", out_path)

Final master dataset saved to: C:\Users\sid24\era5_download\beed_master_2015_2024.csv


: 